In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
import os
os.chdir(r'/workspaces/demo-repo/src/')

import prepare_data, train, preprocess

warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
df = pd.read_csv("/workspaces/demo-repo/data/diabetes.csv")
df.head()

id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80   1,001.00   
1    842517         M        20.57         17.77          132.90   1,326.00   
2  84300903         M        19.69         21.25          130.00   1,203.00   
3  84348301         M        11.42         20.38           77.58     386.10   
4  84358402         M        20.29         14.34          135.10   1,297.00   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0             0.12              0.28            0.30                 0.15   
1             0.08              0.08            0.09                 0.07   
2             0.11              0.16            0.20                 0.13   
3             0.14              0.28            0.24                 0.11   
4             0.10              0.13            0.20                 0.10   

   symmetry_mean  fractal_dimension_mean  radius_se  texture_se  perimeter_se  \
0           0.24                    0.08       1.09        0.91          8.59   
1           0.18                    0.06       0.54        0.73          3.40   
2           0.21                    0.06       0.75        0.79          4.58   
3           0.26                    0.10       0.50        1.16          3.44   
4           0.18                    0.06       0.76        0.78          5.44   

   area_se  smoothness_se  compactness_se  concavity_se  concave points_se  \
0   153.40           0.01            0.05          0.05               0.02   
1    74.08           0.01            0.01          0.02               0.01   
2    94.03           0.01            0.04          0.04               0.02   
3    27.23           0.01            0.07          0.06               0.02   
4    94.44           0.01            0.02          0.06               0.02   

   symmetry_se  fractal_dimension_se  radius_worst  texture_worst  \
0         0.03                  0.01         25.38          17.33   
1         0.01                  0.00         24.99          23.41   
2         0.02                  0.00         23.57          25.53   
3         0.06                  0.01         14.91          26.50   
4         0.02                  0.01         22.54          16.67   

   perimeter_worst  area_worst  smoothness_worst  compactness_worst  \
0           184.60    2,019.00              0.16               0.67   
1           158.80    1,956.00              0.12               0.19   
2           152.50    1,709.00              0.14               0.42   
3            98.87      567.70              0.21               0.87   
4           152.20    1,575.00              0.14               0.20   

   concavity_worst  concave points_worst  symmetry_worst  \
0             0.71                  0.27            0.46   
1             0.24                  0.19            0.28   
2             0.45                  0.24            0.36   
3             0.69                  0.26            0.66   
4             0.40                  0.16            0.24   

   fractal_dimension_worst  Unnamed: 32  
0                     0.12          NaN  
1                     0.09          NaN  
2                     0.09          NaN  
3                     0.17          NaN  
4                     0.08          NaN

In [7]:
df.shape

(768, 9)

In [8]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
config = prepare_data.load_config()

2023-08-25 22:38:07,856 - DEBUG - loading config.yml...


In [11]:
model = 'LogisticRegression'
ds_config = config['diabetes']

In [12]:
ds_config

{'name': 'diabetes',
 'input_file': {'name': 'diabetes.csv',
  'path': '/workspaces/demo-repo/data',
  'drop_nulls': ['outcome']},
 'target': 'outcome',
 'transform': {'glucose': 'MinMaxScaler', 'age': 'log'},
 'models': {'LogisticRegression': {'params': {'C': 1,
    'class_weight': 'balanced',
    'penalty': 'l1',
    'max_iter': 2000,
    'random_state': 42,
    'solver': 'liblinear'},
   'cv': {'penalty': ['l2', 'l1'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'max_iter': [2000, 5000, 10000],
    'class_weight': ['balanced', 'None'],
    'random_state': [42]}}}}

In [14]:
if 'params' in ds_config['models']['LogisticRegression']:
    print(ds_config['models']['LogisticRegression']['params'])

{'C': 1, 'class_weight': 'balanced', 'penalty': 'l1', 'max_iter': 2000, 'random_state': 42, 'solver': 'liblinear'}


In [15]:
df = prepare_data.prepare(model, ds_config)
df.head()

2023-08-25 22:38:48,895 - DEBUG - loading dataset diabetes.csv...
2023-08-25 22:38:48,918 - DEBUG - convert all column names to lowercase...
2023-08-25 22:38:48,923 - DEBUG - dropping NaN from ['outcome']...


pregnancies  glucose  bloodpressure  skinthickness  insulin   bmi  \
0            6      148             72             35        0 33.60   
1            1       85             66             29        0 26.60   
2            8      183             64              0        0 23.30   
3            1       89             66             23       94 28.10   
4            0      137             40             35      168 43.10   

   diabetespedigreefunction  age  outcome  
0                      0.63   50        1  
1                      0.35   31        0  
2                      0.67   32        1  
3                      0.17   21        0  
4                      2.29   33        1

In [16]:
if 'transform' in ds_config:
    df = preprocess.apply_preprocessing(df, model, ds_config)

2023-08-25 22:39:04,502 - DEBUG - performing MinMaxScaler transformation on glucose...
2023-08-25 22:39:04,509 - DEBUG - performing log transformation on age...


In [17]:
train.train_and_predict(df, model, ds_config)

2023-08-25 22:39:24,010 - DEBUG - loading prepared data for training...
2023-08-25 22:39:24,012 - DEBUG - splitting train and test...
2023-08-25 22:39:24,127 - DEBUG - accuracy for LogisticRegression(C=1, class_weight='balanced', max_iter=2000, penalty='l1',
                   random_state=42, solver='liblinear'): 0.8020833333333334
